In [1]:
import pandas as pd
import numpy as np
import warnings
import os
import sys
import urllib.request
import joblib
import simplejson as json
from time import time
warnings.filterwarnings(action='ignore')

import seaborn as sns
sns.set_palette("pastel")
import matplotlib.pylab as plt
from matplotlib import font_manager, rc
%matplotlib inline

import platform
your_os = platform.system()
if your_os == 'Linux':
    rc('font', family='NanumGothic')
elif your_os == 'Windows':
    ttf = "c:/Windows/Fonts/malgun.ttf"
    font_name = font_manager.FontProperties(fname=ttf).get_name()
    rc('font', family=font_name)
elif your_os == 'Darwin':
    rc('font', family='AppleGothic')

In [2]:
os.chdir("../")
root_path = os.getcwd()

# 따옴표 안에 데이터가 있는 폴더의 상위폴더
data_folder_path = os.path.join(root_path, 'data')
original_file_path = os.path.join(data_folder_path, 'original_data')
original_raw_file_path = os.path.join(original_file_path,'raw_data')
original_processed_file_path = os.path.join(original_file_path,'processed_data')

external_file_path = os.path.join(data_folder_path,'external_data')
external_raw_file_path = os.path.join(external_file_path,'raw_data')
external_processed_file_path = os.path.join(external_file_path,'processed_data')

In [3]:
file_list = os.listdir(os.path.join(external_raw_file_path,'local_people'))
local_people = [file for file in file_list if file.startswith("LOCAL")]

In [4]:
temp_df = []
for lp in local_people:
    try:
        temp = pd.read_csv(os.path.join(external_raw_file_path,'local_people',lp),index_col=False)
        temp_df.append(temp)
    except:
        dfcolumns = pd.read_csv(os.path.join(external_raw_file_path,'local_people',lp),nrows = 1)
        temp = pd.read_csv(os.path.join(external_raw_file_path,'local_people',lp), header = None, 
                           skiprows = 1, usecols = list(range(len(dfcolumns.columns))),
                           names = dfcolumns.columns)
        temp_df.append(temp)

In [5]:
cd = pd.read_csv(os.path.join(external_raw_file_path,'행정동코드_매핑정보.csv'),encoding='cp949',header=1)

In [6]:
df = pd.concat(temp_df,ignore_index=True)

In [7]:
df.rename(columns = {'남자0세부터9세생활인구수':'남자_0009세생활인구수'},inplace=True)
df.rename(columns = {'남자70세이상생활인구수':'남자_70세이상생활인구수'},inplace=True)
df.rename(columns = {'여자0세부터9세생활인구수':'여자_0009세생활인구수'},inplace=True)
df.rename(columns = {'여자70세이상생활인구수':'여자_70세이상생활인구수'},inplace=True)

In [8]:
for i in np.arange(1,13):
    temp = list(df.columns[4:])[i][:2]+'_'+list(df.columns[4:])[i][2:4]+list(df.columns[4:])[i][7:]
    name = list(df.columns[4:])[i]
    df.rename(columns={name:temp},inplace=True)

for i in np.arange(15,27):
    temp = list(df.columns[4:])[i][:2]+'_'+list(df.columns[4:])[i][2:4]+list(df.columns[4:])[i][7:]
    name = list(df.columns[4:])[i]
    df.rename(columns={name:temp},inplace=True)

In [9]:
cd['G_DNG_NM'] = cd[['DO_NM','CT_NM','H_DNG_NM']].apply(lambda x: ' '.join(x), axis=1)
GU_DONG = cd[['H_DNG_CD','G_DNG_NM']]
GU_DONG.rename(columns = {'H_DNG_CD':'행정동코드'},inplace=True)

In [10]:
life_pup = pd.merge(df,GU_DONG,on='행정동코드',how='inner')

In [11]:
life_pup = pd.concat([life_pup[life_pup['G_DNG_NM'].str.contains('노원구')], life_pup[life_pup['G_DNG_NM'].str.contains('중구')]])

In [12]:
life_pup.rename(columns = {'기준일ID':'STD_DD'},inplace=True)

In [13]:
life_pup['STD_DD'] = life_pup['STD_DD'].apply(lambda x: pd.to_datetime(str(x), format='%Y%m%d'))

In [14]:
life_pup = life_pup.reset_index(drop=True)

In [15]:
pup0 = life_pup.drop(columns=['행정동코드'])

In [16]:
pup = pd.melt(pup0,id_vars=['STD_DD','시간대구분','G_DNG_NM'],var_name='HUMAN_NM',value_name='FLOW')

In [17]:
pup['SEX_CD'] = np.where(pup['HUMAN_NM'].str.contains('남자'),'M','F')
pup['HDONG_NM'] = pup['G_DNG_NM'].str[6:].apply(lambda x: x.strip())
pup['WEEK_NUM'] = pd.to_datetime(pup['STD_DD']).dt.weekofyear
pup['AGE_CD'] = pup['HUMAN_NM'].str[3:8]
pup['WEEKDAY'] = pd.to_datetime(pup['STD_DD']).dt.weekday

In [18]:
pup1 = pup.drop(columns='HUMAN_NM')

In [19]:
pup

,STD_DD,시간대구분,G_DNG_NM,HUMAN_NM,FLOW,SEX_CD,HDONG_NM,WEEK_NUM,AGE_CD,WEEKDAY
0,2019-02-01,0,서울 노원구 중계2.3동,총생활인구수,41581.1359,F,중계2.3동,5,인구수,4
1,2019-02-01,1,서울 노원구 중계2.3동,총생활인구수,41761.8386,F,중계2.3동,5,인구수,4
2,2019-02-01,2,서울 노원구 중계2.3동,총생활인구수,41844.6152,F,중계2.3동,5,인구수,4
3,2019-02-01,3,서울 노원구 중계2.3동,총생활인구수,41877.3962,F,중계2.3동,5,인구수,4
4,2019-02-01,4,서울 노원구 중계2.3동,총생활인구수,41611.5227,F,중계2.3동,5,인구수,4
...,...,...,...,...,...,...,...,...,...,...
7146523,2020-07-31,19,서울 중구 을지로동,여자_70세이상생활인구수,812.2005,F,을지로동,31,70세이상,4
7146524,2020-07-31,20,서울 중구 을지로동,여자_70세이상생활인구수,665.6440,F,을지로동,31,70세이상,4
7146525,2020-07-31,21,서울 중구 을지로동,여자_70세이상생활인구수,481.1339,F,을지로동,31,70세이상,4
7146526,2020-07-31,22,서울 중구 을지로동,여자_70세이상생활인구수,391.8289,F,을지로동,31,70세이상,4


In [21]:
pup

,STD_DD,시간대구분,G_DNG_NM,HUMAN_NM,FLOW,SEX_CD,HDONG_NM,WEEK_NUM,AGE_CD,WEEKDAY
0,2019-02-01,0,서울 노원구 중계2.3동,총생활인구수,41581.1359,F,중계2.3동,5,인구수,4
1,2019-02-01,1,서울 노원구 중계2.3동,총생활인구수,41761.8386,F,중계2.3동,5,인구수,4
2,2019-02-01,2,서울 노원구 중계2.3동,총생활인구수,41844.6152,F,중계2.3동,5,인구수,4
3,2019-02-01,3,서울 노원구 중계2.3동,총생활인구수,41877.3962,F,중계2.3동,5,인구수,4
4,2019-02-01,4,서울 노원구 중계2.3동,총생활인구수,41611.5227,F,중계2.3동,5,인구수,4
...,...,...,...,...,...,...,...,...,...,...
7146523,2020-07-31,19,서울 중구 을지로동,여자_70세이상생활인구수,812.2005,F,을지로동,31,70세이상,4
7146524,2020-07-31,20,서울 중구 을지로동,여자_70세이상생활인구수,665.6440,F,을지로동,31,70세이상,4
7146525,2020-07-31,21,서울 중구 을지로동,여자_70세이상생활인구수,481.1339,F,을지로동,31,70세이상,4
7146526,2020-07-31,22,서울 중구 을지로동,여자_70세이상생활인구수,391.8289,F,을지로동,31,70세이상,4


In [22]:
life_pup

,STD_DD,시간대구분,행정동코드,총생활인구수,남자_0009세생활인구수,남자_1014세생활인구수,남자_1519세생활인구수,남자_2024세생활인구수,남자_2529세생활인구수,남자_3034세생활인구수,...,여자_3034세생활인구수,여자_3539세생활인구수,여자_4044세생활인구수,여자_4549세생활인구수,여자_5054세생활인구수,여자_5559세생활인구수,여자_6064세생활인구수,여자_6569세생활인구수,여자_70세이상생활인구수,G_DNG_NM
0,2019-02-01,0,11350625,41581.1359,1858.3096,1083.5876,1443.7913,1326.1644,1024.4052,938.7358,...,1174.7337,1513.8010,1724.8761,2064.0048,1553.6693,1557.1557,1602.0804,1163.2484,3954.6800,서울 노원구 중계2.3동
1,2019-02-01,1,11350625,41761.8386,1785.0549,1040.8723,1445.6607,1324.7599,1084.2487,943.8191,...,1187.2831,1534.8230,1712.9536,2101.0812,1548.9966,1562.3745,1583.8154,1180.6795,3940.8582,서울 노원구 중계2.3동
2,2019-02-01,2,11350625,41844.6152,1733.5305,1010.8282,1449.5754,1306.9268,1098.6303,966.5670,...,1186.5000,1522.0234,1719.6099,2103.9860,1545.2247,1584.1685,1563.6674,1171.0468,3955.4783,서울 노원구 중계2.3동
3,2019-02-01,3,11350625,41877.3962,1745.2137,1017.6407,1445.4157,1330.6860,1098.5907,945.1424,...,1159.6564,1534.5133,1725.4864,2117.2065,1527.2999,1585.6936,1561.0936,1172.0026,3917.4873,서울 노원구 중계2.3동
4,2019-02-01,4,11350625,41611.5227,1712.3663,998.4874,1432.0561,1349.6416,1089.0384,940.9494,...,1172.4177,1560.6182,1696.6967,2073.2352,1515.7731,1581.7602,1564.2988,1179.6051,3912.0927,서울 노원구 중계2.3동
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246427,2020-07-31,19,11140605,22224.1748,107.2062,43.6765,93.7226,598.6922,1191.4342,1198.5735,...,1163.8369,809.8865,669.4118,704.4263,564.2407,735.4887,466.7694,369.5454,812.2005,서울 중구 을지로동
246428,2020-07-31,20,11140605,19317.3141,115.1647,46.9188,91.7791,532.1684,1093.2365,1141.5006,...,1040.1940,707.0701,516.8834,584.0771,504.2505,607.4037,410.7243,348.2194,665.6440,서울 중구 을지로동
246429,2020-07-31,21,11140605,15834.4316,103.4800,42.1584,71.6873,477.2416,1075.8233,958.3699,...,941.2018,567.0811,415.7402,465.1078,386.3832,424.8457,320.8749,248.3876,481.1339,서울 중구 을지로동
246430,2020-07-31,22,11140605,12472.2830,98.5195,40.1376,57.6483,418.5848,932.9435,841.5891,...,738.5510,438.0668,332.5112,355.0179,293.3567,289.2451,230.7304,200.4746,391.8289,서울 중구 을지로동


In [23]:
life_pup[life_pup['STD_DD']<'2020-06-01'].to_csv(os.path.join(external_processed_file_path, 'local_people_data.csv'),index=False)
pup[pup['STD_DD']<'2020-06-01'].to_csv(os.path.join(external_processed_file_path, 'local_people_melt.csv'),index=False)
pup1[pup1['STD_DD']>='2020-06-01'].reset_index(drop=True).to_csv(os.path.join(external_processed_file_path, 'local_people_melt_0607.csv'),index=False)